In [144]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [182]:
!ls  ensemble/

cb6133_ln2401_kat2193.prob.npy	     cb6133test_stacker_prob.npy
cb6133test_am4564_hk2903.prob.npy    cb6133test_yw3169_yh3050_newscore.npy
cb6133test_ddl2133_ks3311_.prob.npy  cb6133test_yw3169_yh3050.npy
cb6133test_ps2958_jw3468.prob.npy


In [147]:
yw = np.load('ensemble/cb6133test_yw3169_yh3050.npy') # uses warped list
yw_new = np.load('ensemble/cb6133test_yw3169_yh3050_newscore.npy') # uses warped list

ddl = np.load('ensemble/cb6133test_ddl2133_ks3311_.prob.npy') # uses original label indices

hk = np.load('ensemble/cb6133test_am4564_hk2903.prob.npy') # uses warped list

ln = np.load('ensemble/cb6133_ln2401_kat2193.prob.npy') # uses warped list

ps = np.load('ensemble/cb6133test_ps2958_jw3468.prob.npy') # uses warped list

stack = np.load('ensemble/cb6133test_stacker_prob.npy') # uses warped list

In [152]:
print (yw.shape,yw_new.shape, ddl.shape, hk.shape,ln.shape, ps.shape,stack.shape)

(272, 768, 9) (272, 768, 9) (272, 700, 9) (272, 800, 9) (272, 700, 9) (272, 700, 9) (272, 700, 9)


In [102]:
# warped_order_1 = ['NoSeq', 'H', 'E', 'L','T', 'S', 'G', 'B',  'I']
#                    0        1    2    3   4    5    6    7     8
#                  ['L',     'B', 'E', 'G','I', 'H', 'S', 'T', 'NoSeq'] # new order
fix_warp_list = [8,5,2,0,7,6,3,1,4]

In [103]:
labels = ['L', 'B', 'E', 'G','I', 'H', 'S', 'T', 'NoSeq']

In [130]:
# fix order for hk
ln_fixed = np.zeros_like(ddl)
for count, i in enumerate(fix_warp_list):
    ln_fixed[:,:,i] = ln[:,:700,count] 

In [ ]:
# fix order for hk
hk_fixed = np.zeros_like(ddl)
for count, i in enumerate(fix_warp_list):
    hk_fixed[:,:,i] = hk[:,:700,count] 

In [153]:
# fix order for hk
stack_fixed = np.zeros_like(ddl)
for count, i in enumerate(fix_warp_list):
    stack_fixed[:,:,i] = stack[:,:700,count] 

In [123]:
# fix order for hk
ps_fixed = np.zeros_like(ddl)
for count, i in enumerate(fix_warp_list):
    ps_fixed[:,:,i] = ps[:,:700,count] 

In [107]:
# fix order for yw
yw_fixed = np.zeros_like(ddl)
for count, i in enumerate(fix_warp_list):
    yw_fixed[:,:,i] = yw[:,:700,count] 

In [108]:
yw_new_fixed = np.zeros_like(ddl)
for count, i in enumerate(fix_warp_list):
    yw_new_fixed[:,:,i] = yw_new[:,:700,count] 

In [154]:
def check_softmax(T):
    for i in range(T.shape[0]):
        for j in range(T.shape[1]):
            try:
                assert np.abs(np.sum(T[i,j,:])-1.0)<0.0001
            except:
                print (np.sum(T[i,j,:]), 'failed')
                return 
    print ('outputs are softmaxed')
    

check_softmax(ddl)
check_softmax(hk_fixed)
check_softmax(yw_fixed)
check_softmax(yw_new_fixed)
check_softmax(ps_fixed)
#check_softmax(ln_fixed)
check_softmax(stack_fixed)

outputs are softmaxed
outputs are softmaxed
outputs are softmaxed
outputs are softmaxed
outputs are softmaxed
outputs are softmaxed


In [177]:
summed_probs = ddl+ yw_fixed + hk_fixed + yw_new_fixed + ps_fixed #stack#+ ln

In [178]:
length_list = [len(line.strip().split(',')[2]) for line in open('testing/cb6133test_solution.csv').readlines()]
print ('max protein seq length is', np.max(length_list))

max protein seq length is 632


In [179]:
ensemble_predictions = []
for protein_idx, i in enumerate(length_list):
    new_pred = ''
    for j in range(i):
        new_pred += labels[np.argmax(summed_probs[protein_idx, j ,:])]
    ensemble_predictions.append(new_pred)

In [180]:
with open('ensemble_predictions.csv','w') as f:
    for idx, i in enumerate(ensemble_predictions):
        f.write(str(idx)+','+i + '\n')

In [183]:
# calculating accuracy 
def get_acc(gt,pred):
    assert len(gt)== len(pred)
    correct = 0
    for i in range(len(gt)):
        if gt[i]==pred[i]:
            correct+=1
            
    return (1.0*correct)/len(gt)

gt_all = [line.strip().split(',')[3] for line in open('testing/cb6133test_solution.csv').readlines()]
acc_list = []

for gt,pred in zip(gt_all,ensemble_predictions):
    acc = get_acc(gt,pred)
    acc_list.append(acc)
print ('mean accuracy is', np.mean(acc_list))

mean accuracy is 0.76263035034


In [197]:
name2label = {j:i  for i,j in enumerate(labels[:-1])}
label2name = {i:j  for i,j in enumerate(labels[:-1])}

In [198]:
name2label

{'B': 1, 'E': 2, 'G': 3, 'H': 5, 'I': 4, 'L': 0, 'S': 6, 'T': 7}

In [210]:
label2name

{0: 'L', 1: 'B', 2: 'E', 3: 'G', 4: 'I', 5: 'H', 6: 'S', 7: 'T'}

In [199]:
classes = len(labels[:-1])
conf_matrix = np.zeros((classes,classes))
for gt,pred in zip(gt_all,ensemble_predictions):
    for g,p in zip(gt,pred):
        conf_matrix[name2label[p],name2label[g]]+=1.0

In [200]:
recall_list = np.zeros((classes,))
precision_list = np.zeros((classes,))
for i in range(classes):
    if np.sum(conf_matrix[:,i])!=0:
        recall_list[i] = conf_matrix[i,i]/float(np.sum(conf_matrix[:,i]))
    else:
        recall_list[i] = 0 

    if np.sum(conf_matrix[i,:])!=0:
        precision_list[i] = conf_matrix[i,i]/float(np.sum(conf_matrix[i,:]))
    else:
        precision_list[i] = 0 

fscore = np.zeros((classes,))

for i in range(classes):
    if precision_list[i] + recall_list[i]!=0:
        fscore[i] = 2.0 *precision_list[i] * recall_list[i] / (precision_list[i] + recall_list[i])
    else:
        fscore[i] = 0

In [205]:
for i in range(classes):
    print ('precision of class',label2name[i], precision_list[i])
print ('average precision', np.mean(precision_list))

precision of class L 0.588216119306
precision of class B 0.666666666667
precision of class E 0.807053132558
precision of class G 0.545191612437
precision of class I 0.0
precision of class H 0.866672721493
precision of class S 0.567415730337
precision of class T 0.588967691095
average precision 0.578772959237


In [206]:
for i in range(classes):
    print ('recall of class',label2name[i], recall_list[i])
print ('average recall', np.mean(recall_list))

recall of class L 0.685600303951
recall of class B 0.073482428115
recall of class E 0.846550454211
recall of class G 0.327541268462
recall of class I 0.0
recall of class H 0.943820780377
recall of class S 0.224944320713
recall of class T 0.593174603175
average recall 0.461889269875


In [208]:
for i in range(classes):
    print ('fscore of class',label2name[i], fscore[i])
print ('average f-score', np.mean(fscore))

fscore of class L 0.633185666038
fscore of class B 0.132374100719
fscore of class E 0.826330084678
fscore of class G 0.409226594301
fscore of class I 0.0
fscore of class H 0.903603049098
fscore of class S 0.322169059011
fscore of class T 0.591063661526
average f-score 0.477244026922


In [215]:
with open('metrics_cb6133.csv', 'w') as f:
    f.write('secondary structure, precision, recall, f-score' + '\n')
    for i in range(classes):
        f.write(str(label2name[i]) + ',' +str(precision_list[i]) + ',' + str(recall_list[i]) + ',' +  str(fscore[i]) + '\n')
        
    f.write('mean,'+str(np.mean(precision_list)) +','+ str(np.mean(recall_list))+','+str(np.mean(fscore)) + '\n'    )